# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd3fc461490>
├── 'a' --> tensor([[0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552]])
└── 'x' --> <FastTreeValue 0x7fd3fc461a30>
    └── 'c' --> tensor([[ 1.5351, -0.8296, -1.2411,  0.9635],
                        [-0.3917, -0.2145,  0.6835,  2.1358],
                        [-1.1314, -0.6800,  1.0157, -1.0037]])

In [4]:
t.a

tensor([[0.7637, 1.4298, 1.0616],
        [0.9079, 0.7305, 0.4552]])

In [5]:
%timeit t.a

74.3 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd3fc461490>
├── 'a' --> tensor([[ 2.2998, -0.4478,  0.9933],
│                   [-0.7404, -1.0409,  0.9346]])
└── 'x' --> <FastTreeValue 0x7fd3fc461a30>
    └── 'c' --> tensor([[ 1.5351, -0.8296, -1.2411,  0.9635],
                        [-0.3917, -0.2145,  0.6835,  2.1358],
                        [-1.1314, -0.6800,  1.0157, -1.0037]])

In [7]:
%timeit t.a = new_value

81.7 ns ± 0.282 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[0.7637, 1.4298, 1.0616],
               [0.9079, 0.7305, 0.4552]]),
    x: Batch(
           c: tensor([[ 1.5351, -0.8296, -1.2411,  0.9635],
                      [-0.3917, -0.2145,  0.6835,  2.1358],
                      [-1.1314, -0.6800,  1.0157, -1.0037]]),
       ),
)

In [10]:
b.a

tensor([[0.7637, 1.4298, 1.0616],
        [0.9079, 0.7305, 0.4552]])

In [11]:
%timeit b.a

70.3 ns ± 0.384 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1414, -0.9350,  0.0998],
               [-1.0774,  0.0107,  0.7316]]),
    x: Batch(
           c: tensor([[ 1.5351, -0.8296, -1.2411,  0.9635],
                      [-0.3917, -0.2145,  0.6835,  2.1358],
                      [-1.1314, -0.6800,  1.0157, -1.0037]]),
       ),
)

In [13]:
%timeit b.a = new_value

586 ns ± 1.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

972 ns ± 3.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 31.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 777 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd345762a60>
├── 'a' --> tensor([[[0.7637, 1.4298, 1.0616],
│                    [0.9079, 0.7305, 0.4552]],
│           
│                   [[0.7637, 1.4298, 1.0616],
│                    [0.9079, 0.7305, 0.4552]],
│           
│                   [[0.7637, 1.4298, 1.0616],
│                    [0.9079, 0.7305, 0.4552]],
│           
│                   [[0.7637, 1.4298, 1.0616],
│                    [0.9079, 0.7305, 0.4552]],
│           
│                   [[0.7637, 1.4298, 1.0616],
│                    [0.9079, 0.7305, 0.4552]],
│           
│                   [[0.7637, 1.4298, 1.0616],
│                    [0.9079, 0.7305, 0.4552]],
│           
│                   [[0.7637, 1.4298, 1.0616],
│                    [0.9079, 0.7305, 0.4552]],
│           
│                   [[0.7637, 1.4298, 1.0616],
│                    [0.9079, 0.7305, 0.4552]]])
└── 'x' --> <FastTreeValue 0x7fd3fc4cdd30>
    └── 'c' --> tensor([[[ 1.5351, -0.8296, -1.2411,  0.9635],
           

In [21]:
%timeit t_stack(trees)

37.3 µs ± 204 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd34c375cd0>
├── 'a' --> tensor([[0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552],
│                   [0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552],
│                   [0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552],
│                   [0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552],
│                   [0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552],
│                   [0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552],
│                   [0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552],
│                   [0.7637, 1.4298, 1.0616],
│                   [0.9079, 0.7305, 0.4552]])
└── 'x' --> <FastTreeValue 0x7fd34576bb50>
    └── 'c' --> tensor([[ 1.5351, -0.8296, -1.2411,  0.9635],
                        [-0.3917, -0.2145,  0.6835,  2.1358],
                        [-1.1314, -0.6800,  1.0157, -1.0037],
   

In [23]:
%timeit t_cat(trees)

35.8 µs ± 687 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.7 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.5351, -0.8296, -1.2411,  0.9635],
                       [-0.3917, -0.2145,  0.6835,  2.1358],
                       [-1.1314, -0.6800,  1.0157, -1.0037]],
              
                      [[ 1.5351, -0.8296, -1.2411,  0.9635],
                       [-0.3917, -0.2145,  0.6835,  2.1358],
                       [-1.1314, -0.6800,  1.0157, -1.0037]],
              
                      [[ 1.5351, -0.8296, -1.2411,  0.9635],
                       [-0.3917, -0.2145,  0.6835,  2.1358],
                       [-1.1314, -0.6800,  1.0157, -1.0037]],
              
                      [[ 1.5351, -0.8296, -1.2411,  0.9635],
                       [-0.3917, -0.2145,  0.6835,  2.1358],
                       [-1.1314, -0.6800,  1.0157, -1.0037]],
              
                      [[ 1.5351, -0.8296, -1.2411,  0.9635],
                       [-0.3917, -0.2145,  0.6835,  2.1358],
                       [-1.1314, -0.6800,  1.0157, -1.0037]],

In [26]:
%timeit Batch.stack(batches)

88.5 µs ± 452 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.5351, -0.8296, -1.2411,  0.9635],
                      [-0.3917, -0.2145,  0.6835,  2.1358],
                      [-1.1314, -0.6800,  1.0157, -1.0037],
                      [ 1.5351, -0.8296, -1.2411,  0.9635],
                      [-0.3917, -0.2145,  0.6835,  2.1358],
                      [-1.1314, -0.6800,  1.0157, -1.0037],
                      [ 1.5351, -0.8296, -1.2411,  0.9635],
                      [-0.3917, -0.2145,  0.6835,  2.1358],
                      [-1.1314, -0.6800,  1.0157, -1.0037],
                      [ 1.5351, -0.8296, -1.2411,  0.9635],
                      [-0.3917, -0.2145,  0.6835,  2.1358],
                      [-1.1314, -0.6800,  1.0157, -1.0037],
                      [ 1.5351, -0.8296, -1.2411,  0.9635],
                      [-0.3917, -0.2145,  0.6835,  2.1358],
                      [-1.1314, -0.6800,  1.0157, -1.0037],
                      [ 1.5351, -0.8296, -1.2411,  0.9635],
                   

In [28]:
%timeit Batch.cat(batches)

166 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

380 µs ± 7.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
